# Réseaux Navigocorpus

In [1]:
import networkx as nx
from poitousprint import Portic

In [10]:
INNER_ADMIRALTIES = {'La Rochelle', 'Marennes', 'Sables-d’Olonne'}
INNER_PROVINCES = {'Aunis', 'Poitou', 'Saintonge'}

In [3]:
client = Portic()

In [4]:
flows = client.get_flows(year=1789, source_subset='Poitou_1789')
flows = [flow for flow in flows if flow['departure_action'] == 'Out']

In [11]:
pointcalls = client.get_pointcalls(year=1789)
pointcalls = [pc for pc in pointcalls if pc['pointcall_province'] in INNER_PROVINCES]

In [12]:
pointcalls[0]

{'pkid': 85850,
 'record_id': '00342128',
 'pointcall': 'La Rochelle',
 'pointcall_uhgs_id': 'A0198999',
 'toponyme_fr': 'La Rochelle',
 'toponyme_en': 'La Rochelle',
 'latitude': '46.166667',
 'longitude': '-1.15',
 'pointcall_admiralty': 'La Rochelle',
 'pointcall_province': 'Aunis',
 'pointcall_states': '[{"1749-1815" : "France"}]',
 'pointcall_substates': None,
 'pointcall_states_en': '[{"1749-1815" : "France"}]',
 'pointcall_substates_en': None,
 'state_1789_fr': 'France',
 'state_1789_en': 'France',
 'substate_1789_fr': None,
 'substate_1789_en': None,
 'nb_conges_1787_inputdone': 1221.0,
 'nb_conges_1787_cr': None,
 'nb_conges_1789_inputdone': 1060.0,
 'nb_conges_1789_cr': None,
 'pointcall_status_uncertainity': 0.0,
 'pointcall_status': 'siège amirauté',
 'shiparea': 'ACE-ROCH',
 'pointcall_point': '0101000020110F000054C16EA11641FFC0ACD86784FE265641',
 'ferme_direction': 'La Rochelle',
 'ferme_bureau': 'La Rochelle',
 'ferme_bureau_uncertainty': 0.0,
 'partner_balance_1789': No

In [5]:
tonnage_classes = sorted(set(flow['tonnage_class'] for flow in flows), key=lambda x: int(x[1:].split('-')[0]))
tonnage_classes

['[1-20]', '[21-50]', '[51-100]', '[101-200]', '[201-500]']

## Réseau macro

In [6]:
def build_graph(name, tonnage_class=None):
    graph = nx.DiGraph()

    def add_node(g, name, admiralty=None, peche=0):
        if admiralty is None:
            admiralty = 'n/a'

        if g.has_node(name):
            g.nodes[name]['peche'] += peche
        else:
            g.add_node(
                name,
                admiralty=admiralty,
                peche=peche,
                in_region=admiralty in INNER_ADMIRALTIES,
                inside_degree=0,
                outside_degree=0
            )

    def add_edge(g, source, target, tonnage):
        if g.has_edge(source, target):
            attr = g[source][target]
            attr['weight'] += 1
            attr['tonnage'] += tonnage
        else:
            g.add_edge(
                source,
                target,
                weight=1,
                tonnage=tonnage
            )

    for flow in flows:
        if tonnage_class is not None and flow['tonnage_class'] != tonnage_class:
            continue
        
        source = flow['departure']
        target = flow['destination']

        source_admiralty = flow['departure_admiralty']
        target_admiralty = flow['destination_admiralty']
        
        tonnage = 0

        try:
            tonnage = int(flow['tonnage'])
        except ValueError:
            pass

        inside_flow = source_admiralty in INNER_ADMIRALTIES and target_admiralty in INNER_ADMIRALTIES

        # Macro graph
        if source == target:
            add_node(graph, source, source_admiralty, 1)
        else:
            add_node(graph, source, source_admiralty)
            add_node(graph, target, target_admiralty)
            add_edge(graph, source, target, tonnage)

            if source_admiralty in INNER_ADMIRALTIES:
                if inside_flow:
                    graph.nodes[source]['inside_degree'] += 1
                else:
                    graph.nodes[source]['outside_degree'] += 1
    
    nx.write_gexf(graph, '%s.gexf' % name)

In [7]:
build_graph('macro')

In [8]:
for cls in tonnage_classes:
    build_graph(cls, tonnage_class=cls)